# 认识Bedrock

![](images/bedrock_models.png)

In [85]:
# !pip install -r requirements.txt

### boto3创建 `bedrock` client

In [1]:
import json
import boto3
import base64

# 指定要使用的配置文件名称
profile_name = 'new'

# 创建 AWS 客户端，并指定配置文件名称
session = boto3.Session(profile_name=profile_name)


#Create the connection to Bedrock
bedrock = session.client(
    service_name='bedrock',
    region_name='us-west-2', 
)

bedrock_runtime = session.client(
    service_name='bedrock-runtime',
    region_name='us-west-2', 
    
)

### 列出可用模型版本

In [98]:
# List all foundation models
all_llms = [ model['modelId'] for model in bedrock.list_foundation_models()['modelSummaries']]
all_llms

['amazon.titan-tg1-large',
 'amazon.titan-embed-g1-text-02',
 'amazon.titan-text-lite-v1:0:4k',
 'amazon.titan-text-lite-v1',
 'amazon.titan-text-express-v1:0:8k',
 'amazon.titan-text-express-v1',
 'amazon.titan-embed-text-v1:2:8k',
 'amazon.titan-embed-text-v1',
 'amazon.titan-embed-image-v1:0',
 'amazon.titan-embed-image-v1',
 'amazon.titan-image-generator-v1:0',
 'amazon.titan-image-generator-v1',
 'stability.stable-diffusion-xl',
 'stability.stable-diffusion-xl-v0',
 'stability.stable-diffusion-xl-v1:0',
 'stability.stable-diffusion-xl-v1',
 'ai21.j2-grande-instruct',
 'ai21.j2-jumbo-instruct',
 'ai21.j2-mid',
 'ai21.j2-mid-v1',
 'ai21.j2-ultra',
 'ai21.j2-ultra-v1',
 'anthropic.claude-instant-v1:2:100k',
 'anthropic.claude-instant-v1',
 'anthropic.claude-v2:0:18k',
 'anthropic.claude-v2:0:100k',
 'anthropic.claude-v2:1:18k',
 'anthropic.claude-v2:1:200k',
 'anthropic.claude-v2:1',
 'anthropic.claude-v2',
 'anthropic.claude-3-sonnet-20240229-v1:0:28k',
 'anthropic.claude-3-sonnet-2

### `Anthropic (Claude v3 (Opus))`

In [83]:
# Claude v3 (Opus)有强大的逻辑推理与解题能力

In [5]:
prompt_data = """小圆的半径是3cm,大圆的半径是5cm,大圆与小圆的面积比是()。
A.3:5
B.9:25
C.25:9
D.5:3"""

body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_data
                        }
                    ]
                }
            ],
        }

body = json.dumps(body) # Encode body as JSON string

model_id = "anthropic.claude-3-opus-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

In [6]:
response = bedrock_runtime.invoke_model(body=body,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)

In [7]:
response_body = json.loads(response.get("body").read())
for output in response_body.get("content", []):
    print(output["text"])

在这个问题中,我们需要找出大圆与小圆面积的比值。让我们一步步来解决:

1. 首先,我们知道圆的面积公式是:A = πr², 其中A是面积,r是半径。

2. 小圆的半径是3cm,所以小圆的面积是:
   A₁ = π(3²) = 9π cm²

3. 大圆的半径是5cm,所以大圆的面积是:
   A₂ = π(5²) = 25π cm²

4. 现在,我们可以计算大圆与小圆面积的比值:
   A₂ : A₁ = 25π : 9π

5. 我们可以约分π,得到:
   25 : 9

所以,正确答案是选项B: 25 : 9,也可以写作25/9或2.78 (保留两位小数)。


### With `Mistral (mixtral-8x7b)`

In [ ]:
# Mistral (mixtral-8x7b) 是社区模型中performance突出的模型，这里我们使用mixtral-8x7b

In [100]:
prompt_data = """写一个德语B1的学习大纲"""

body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_data
                        }
                    ]
                }
            ],
        }

body = json.dumps(body) # Encode body as JSON string

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                        modelId=modelId, 
                                        accept=accept, 
                                        contentType=contentType)

response_body = json.loads(response.get('body').read().decode('utf-8'))

for output in response_body.get("content", []):
    print(output["text"])


以下是一个德语B1水平的学习大纲,供参考:

一、语法 (Grammatik)
1. 复习A1-A2级语法知识
2. 时态:现在完成时,过去完成时,将来时
3. 虚拟语气(Konjunktiv II)
4. 被动语态(Passiv)
5. 关系从句(Relativsätze)
6. 情态动词的过去式和过去完成时
7. 带zu不定式
8. 第二虚拟式

二、词汇 (Wortschatz)
1. 扩充日常生活、工作、学习、旅游等主题的词汇量
2. 常见词汇搭配和用法
3. 常见缩略词和习语
4. 近义词和反义词的区分

三、阅读 (Lesen)
1. 阅读中等难度的文章,如新闻报道、个人博客、简单的文学作品等
2. 理解文章主旨,把握关键信息
3. 推断生词词义,理解句子结构

四、听力 (Hören)
1. 听懂日常对话、广播节目、电视新闻等
2. 理解说话者的观点和态度
3. 听写练习,提高听力理解和笔记能力

五、口语 (Sprechen)
1. 就日常话题进行对话和讨论,表达个人观点
2. 描述个人经历,讲述简单故事
3. 进行角色扮演,模拟日常情景
4. 练习德语发音和语调

六、写作 (Schreiben)
1. 写作个人信件、电子邮件
2. 描述图表、图像
3. 撰写短文,如记叙文、议论文等
4. 书面总结听到或读到的信息

七、文化 (Kultur)
1. 了解德国的地理、历史、政治等基本知识
2. 了解德国的生活方式、习俗和价值观
3. 学习德国文学、音乐、艺术等

八、考试准备 (Prüfungsvorbereitung)
1. 了解德语B1考试的考试形式和评分标准
2. 进行模拟考试,熟悉考试流程
3. 针对薄弱环节加强训练,提高考试成绩

注意:以上学习大纲仅供参考,具体学习计划需根据个人情况灵活调整。持之以恒的练习和良好的学习习惯是提高德语水平的关键。祝学习顺利!


In [97]:
for output in response_body.get("content", []):
    print(output["text"])

The word that best fits the blank is "egalitarian".

Egalitarian means believing in or based on the principle that all people are equal and deserve equal rights and opportunities. If Clark was egalitarian as a theater director, it suggests he treated everyone in the cast and crew as equals, rather than being an authoritarian who demanded obedience and maintained strict hierarchies.

The other options don't fit as well in the context:

- Histrionic means excessively dramatic or emotional, which doesn't directly contrast with being authoritarian.
- Megalomaniacal means having delusions of grandeur or an obsessive desire for power, which aligns more with authoritarianism than contrasts with it.
- Indolent means lazy, which isn't a clear opposite of authoritarian.
- Charismatic means charming and able to inspire devotion in others, which, while a positive trait, doesn't necessarily imply a contrast with authoritarianism.

Therefore, "egalitarian" is the best choice to fill in the blank, as

### 多模态图片输入 `Anthropic Claude v3 Haiku`

In [84]:
# Claude v3 Haiku成本极低。推理很快，是支持多模态输入的性价比最高的模型

In [5]:
with open("images/math.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')

payload = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": base64_string
                        }
                    },
                    {
                        "type": "text",
                        "text": "As shown in the figure, in the rectangle quadrilateral ABCD, if E is a point on the edge AB, DAE folded along the crease DE to DA'E, Please independently assess the correctness of the following three statements and provide a complete analysis: \n1. The trajectory of the projection of point A' is F, AF is perpendicular to DE;\n2. AF intersects DE at O, When E varies, the trajectory of O is a circle with AD as the diameter;\n3. The trajectory of point A' is a circle with O as the center and AO as the radius;"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
model_id = "anthropic.claude-3-opus-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body_bytes,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)


In [6]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('content')[0].get('text'))

Let's analyze each statement independently:

1. "The trajectory of the projection of point A' is F, AF is perpendicular to DE."
This statement is correct. As the point E moves along AB, the folding line DE also moves. The point A' is the reflection of point A across the folding line DE. By the properties of reflections, the line segment AA' is always perpendicular to the folding line DE, and its midpoint lies on DE. The trajectory of point F, which is the projection of A' onto AD, forms a line segment perpendicular to DE.

2. "AF intersects DE at O. When E varies, the trajectory of O is a circle with AD as the diameter."
This statement is correct. Let's prove it using the inscribed angle theorem. As E moves along AB, the angle ∠ADE remains a right angle (90°). Therefore, points A, D, and E always form a right triangle with ∠ADE = 90°. The point O, being the intersection of AF and DE, is the midpoint of the hypotenuse AD in this right triangle. According to the inscribed angle theorem, 